# Node2vec

In [1]:
!pip install node2vec

In [2]:
import networkx as nx
import os
import xml.etree.ElementTree as ET
from node2vec import Node2Vec
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
import nltk
import string
import spacy
from operator import itemgetter
nlp = spacy.load('en_core_web_sm')

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
directorio = "/content/drive/MyDrive/PLN_UADY/"

### Generar Grafos

In [5]:
"""funcion para generar grafos de asociacion y frecuencia a partir del xml EAT"""

def lee_xml_eat():
    lista_grafos = []
    G_freq=nx.Graph()
    G_asoc= nx.Graph()
    conjunto_nodos = set()
    tree = ET.parse(directorio+'eat-stimulus-response.xml')
    root = tree.getroot()
    todos = 8211
    i=0
    nodos={}
    total = 0
    j = 0
    for child in root:
        estimulo = child.get('word')
        total =child.get('all')
        estimulo = estimulo.lower()
        estimulo = estimulo.replace(" ","_")
        if estimulo not in conjunto_nodos:
            nodos[estimulo] = j
            G_freq.add_node(estimulo)
            G_asoc.add_node(estimulo)
            conjunto_nodos.add(estimulo)
            j+=1
        i+=1
        #print(i,"de",todos)
        for response in child.iter('response'):
            frecuencia = response.get('n')
            asociacion = response.get('r')
            respuesta = response.get('word')
            respuesta = respuesta.lower()
            respuesta = respuesta.replace(" ","_")
            if respuesta not in conjunto_nodos:
                nodos[respuesta] = j
                G_freq.add_node(respuesta)
                G_asoc.add_node(respuesta)
                conjunto_nodos.add(respuesta)
                j+=1
            G_freq.add_edge(estimulo,respuesta,weight=float(1/int(frecuencia)))
            G_asoc.add_edge(estimulo,respuesta,weight=float(1-float(asociacion)))
    lista_grafos.append(G_freq)
    lista_grafos.append(G_asoc)
    print("Grafos creados")
    return lista_grafos

### Guardar node embeddings

In [ ]:
"""grafos = lee_xml_eat()
grafo_freq = grafos[0]
grafo_asoc = grafos[1]
node2vec = Node2Vec(grafo_freq, dimensions=300)
model = node2vec.fit(window=10, min_count=1, batch_words=4)
print("Guardando embeddings")
model.wv.save_word2vec_format(directorio+"freq.emb")
print("Guardado")"""

'grafos = lee_xml_eat()\ngrafo_freq = grafos[0]\ngrafo_asoc = grafos[1]\nnode2vec = Node2Vec(grafo_freq, dimensions=300)\nmodel = node2vec.fit(window=10, min_count=1, batch_words=4)\nprint("Guardando embeddings")\nmodel.wv.save_word2vec_format(directorio+"freq.emb")\nprint("Guardado")'

In [ ]:
"""node2vec = Node2Vec(grafo_asoc, dimensions=300)
model = node2vec.fit(window=10, min_count=1, batch_words=4)
#batch = numero de palabras que va a tomar para cada bloque de entrenamiento
print("Guardando embeddings")
model.wv.save_word2vec_format(directorio+"asoc.emb")
print("Guardado")"""

'node2vec = Node2Vec(grafo_asoc, dimensions=300)\nmodel = node2vec.fit(window=10, min_count=1, batch_words=4)\n#batch = numero de palabras que va a tomar para cada bloque de entrenamiento\nprint("Guardando embeddings")\nmodel.wv.save_word2vec_format(directorio+"asoc.emb")\nprint("Guardado")'

### Test Asociacion

In [6]:
from gensim.models import KeyedVectors
from gensim.test.utils import datapath

wv = KeyedVectors.load_word2vec_format(directorio+"asoc.emb", binary=False)
total = wv["king"] - wv["man"] + wv["woman"]
print("Modelo Cargado...")
print(wv.most_similar(positive=[total], topn=20))

Modelo Cargado...
[('king', 0.7085300087928772), ('woman', 0.5515092015266418), ('throne', 0.38645023107528687), ('majesty', 0.35767078399658203), ('andrews', 0.3566025197505951), ('princessling', 0.3476901054382324), ('abducted', 0.3460976779460907), ('the_queen', 0.34567052125930786), ('kingly', 0.34363603591918945), ('zonophone', 0.33773913979530334), ('pageantry', 0.3359178304672241), ('crown', 0.32742375135421753), ('liver_salt', 0.3254445195198059), ('monarch', 0.3246691823005676), ('liver_salts', 0.32320883870124817), ('queen', 0.3220468759536743), ('reigning', 0.31814107298851013), ('dethrone', 0.3175676763057709), ('robe', 0.3145774304866791), ('hong', 0.3141893148422241)]


In [ ]:
wv = KeyedVectors.load_word2vec_format(directorio+"asoc.emb", binary=False)
total = wv["paris"] - wv["france"] + wv["madrid"]
print("Modelo Cargado...")
print(wv.most_similar(positive=[total], topn=20))

Modelo Cargado...
[('madrid', 0.7264711856842041), ('paris', 0.538062334060669), ('groove', 0.4280698597431183), ('seine', 0.3851071894168854), ('spain', 0.34135061502456665), ('68', 0.3284861147403717), ('rave', 0.3217294216156006), ('sightseeing', 0.31777504086494446), ('metro', 0.3159087598323822), ('groovy', 0.3110942840576172), ('real', 0.3089691400527954), ('eiffel', 0.3074709475040436), ('without-it', 0.29267656803131104), ('bonn', 0.2753206789493561), ('furrow', 0.2730983793735504), ('mode', 0.2690190374851227), ('eiffel_tower', 0.26821866631507874), ('fashion', 0.26809167861938477), ('styled', 0.2678219974040985), ('portugal', 0.26322516798973083)]


### Test Frecuencia

In [ ]:
wv = KeyedVectors.load_word2vec_format(directorio+"freq.emb", binary=False)
total = wv["king"] - wv["man"] + wv["woman"]
print("Modelo Cargado...")
print(wv.most_similar(positive=[total], topn=20))
total = wv["france"] - wv["paris"]+  wv["madrid"]
print("Modelo Cargado...")
print(wv.most_similar(positive=[total], topn=20))

Modelo Cargado...
[('king', 0.6755074262619019), ('woman', 0.5504201054573059), ('byzantine', 0.3673710227012634), ('hong', 0.3349175751209259), ('princess', 0.3269363343715668), ('andrews', 0.3264494836330414), ('raymond', 0.3119525909423828), ('and_queens', 0.30564355850219727), ('shoe-shop', 0.30482596158981323), ('footstool', 0.3004297912120819), ('liver_salt', 0.3001447021961212), ('abducted', 0.2959466874599457), ('robe', 0.2950398325920105), ('zonophone', 0.2947472631931305), ('throne', 0.2942342162132263), ('fourth', 0.29377779364585876), ('courts', 0.2904150187969208), ('kingly', 0.2900291085243225), ('orb', 0.28961381316185), ('monastery', 0.28867971897125244)]
Modelo Cargado...
[('madrid', 0.7211812138557434), ('france', 0.5648890733718872), ('spain', 0.4949127733707428), ('lyon', 0.44699403643608093), ('groove', 0.3758266270160675), ('francs', 0.3514481484889984), ('souvenirs', 0.34136536717414856), ('niche', 0.3394327759742737), ('froggy', 0.30094054341316223), ('portugal'

### Diccionario Inverso

In [ ]:
# Target: Bee
wv = KeyedVectors.load_word2vec_format(directorio+"asoc.emb", binary=False)
total = wv["insect"] + wv["honey"] + wv["stripes"] + wv["yellow"] +wv["black"]
print("Modelo Cargado...")
results = wv.most_similar([total], topn=20)
results = [element for element in results if element[0] not in ["insect","honey","stripes","yellow","black"]]
print(results)

Modelo Cargado...
[('bumble-bee', 0.4837120771408081), ('tsetse', 0.4761532247066498), ('mosquitoes', 0.46379515528678894), ('insects', 0.46071210503578186), ('wasp', 0.4518756866455078), ('sting', 0.4446922540664673), ('stripe', 0.4429231286048889), ('caterpillar', 0.43826642632484436), ('mosquito', 0.4308270215988159), ('buzz', 0.4273468852043152), ('stung', 0.420186847448349), ('moth', 0.4174724519252777), ('colour', 0.4158550202846527), ('zebra', 0.4125511944293976), ('bugs', 0.41218772530555725)]


### Precision at K

In [7]:
import nltk
import spacy
import string
import os

In [8]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [9]:
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 81.7 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [10]:
nlp = spacy.load("en_core_web_sm")

In [11]:
def limpia_lematiza(cadena):
    limpiado = ""
    palabras_funcionales = nltk.corpus.stopwords.words('english')
    # Quita los signos de puntuacion
    for c in string.punctuation:
      cadena = cadena.replace(c," ")
    # Elimino espacios extra
    cadena = cadena.strip()
    # Itero cada palabra en la string
    for palabra in cadena.split():
      if palabra not in palabras_funcionales:
        # Lematizo
        doc = nlp(palabra)
        limpiado += doc[0].lemma_ + " "
    return limpiado.rstrip()

In [12]:
def precision(k,arreglo,concepto):
  tope = k
  encontrado = 0
  if k> len(arreglo):
    tope = len(arreglo)
  for i in range (0,tope):
    if arreglo[i] == concepto.strip():
      encontrado = 1
  return encontrado

In [13]:
dir = "/content/drive/MyDrive/PLN_UADY/concepts"

In [14]:
directorio = "/content/drive/MyDrive/PLN_UADY/"

In [32]:
def rendimiento_node2vec_asoc():
  wv = KeyedVectors.load_word2vec_format(directorio+"asoc.emb", binary=False)
  total_as, p1, p3, p5 = 0,0,0,0
  for filename in os.listdir(dir):
    f = os.path.join(dir, filename)
    concept = str(filename)[:-4].lower().strip()
    print(f"TARGET: {concept}\n")
    file = open(f, "r")
    lines = file.readlines()
    for line in lines:
      print(f"Definicion:\t{line}")
      line = limpia_lematiza(line.lower())
      total = 0
      tokens = line.split()
      tokens = [t for t in tokens if t in wv]
      for token in tokens:
        total += wv[token]
      results = wv.most_similar([total], topn=20)
      results = [element for element in results if element[0] not in tokens]
      print(f"Results:\t{results}")
      candidatos = [element[0] for element in results]
      p1 += precision(1, candidatos, concept)
      p3 += precision(3, candidatos, concept)
      p5 += precision(5, candidatos, concept)
      total_as += 1
  print("Asociacion")
  print("p@1:",p1/total_as,"p@3:",p3/total_as,"p@5:",p5/total_as)



In [33]:
def rendimiento_node2vec_freq():
  wv = KeyedVectors.load_word2vec_format(directorio+"freq.emb", binary=False)
  total_as, p1, p3, p5 = 0,0,0,0
  for filename in os.listdir(dir):
    f = os.path.join(dir, filename)
    concept = str(filename)[:-4].lower().strip()
    print(f"TARGET: {concept}\n")
    file = open(f, "r")
    lines = file.readlines()
    for line in lines:
      print(f"Definicion:\t{line}")
      line = limpia_lematiza(line.lower())
      total = 0
      tokens = line.split()
      tokens = [t for t in tokens if t in wv]
      for token in tokens:
        total += wv[token]
      results = wv.most_similar([total], topn=20)
      results = [element for element in results if element[0] not in tokens]
      print(f"Results:\t{results}")
      candidatos = [element[0] for element in results]
      p1 += precision(1, candidatos, concept)
      p3 += precision(3, candidatos, concept)
      p5 += precision(5, candidatos, concept)
      total_as += 1
  print("Frecuencia")
  print("p@1:",p1/total_as,"p@3:",p3/total_as,"p@5:",p5/total_as)

In [34]:
rendimiento_node2vec_asoc()

TARGET: squirrel

Definicion:	It’s a little rodent and can be red or grey, it has a big bushy tail  

Results:	[('squirrel', 0.5697661638259888), ('wangler', 0.480981707572937), ('fox', 0.46797892451286316), ('brock', 0.45514923334121704), ('guinea-pig', 0.4461306929588318), ('shrew', 0.4450259804725647), ('elizabeth_taylor', 0.44498249888420105), ('ceros', 0.4441850483417511), ('two_cities', 0.4409492015838623), ('animal', 0.4368653893470764), ('hamster', 0.4332946240901947), ('mice', 0.4305364787578583), ('bitches', 0.4297550916671753)]
Definicion:	A small rodent living in trees with a long bushy tail  

Results:	[('squirrel', 0.5700526833534241), ('guinea-pig', 0.4599015414714813), ('rabbit', 0.4550666809082031), ('shrew', 0.4548138976097107), ('brock', 0.45326724648475647), ('furry', 0.4528180956840515), ('hamster', 0.4513764977455139), ('two_cities', 0.44500067830085754), ('elizabeth_taylor', 0.442573606967926), ('animal', 0.44130590558052063), ('wangler', 0.43633124232292175), ('

In [35]:
rendimiento_node2vec_freq()

TARGET: squirrel

Definicion:	It’s a little rodent and can be red or grey, it has a big bushy tail  

Results:	[('squirrel', 0.4905562400817871), ('taming', 0.4808814823627472), ('bushel', 0.45331451296806335), ('catfish', 0.4518391788005829), ('mammal', 0.444499135017395), ('bitches', 0.4432355463504791), ('preying', 0.4372878074645996), ('pecked', 0.4369546175003052), ('cetera', 0.4363158345222473), ('guinea-pig', 0.43627962470054626), ('fox', 0.4358427822589874), ('ceros', 0.4353017807006836), ('latitudinal', 0.4285621643066406), ('bolognese', 0.4282897710800171)]
Definicion:	A small rodent living in trees with a long bushy tail  

Results:	[('squirrel', 0.5014120936393738), ('alligator', 0.47332119941711426), ('taming', 0.4662151038646698), ('bolognese', 0.4613943099975586), ('two_cities', 0.44606927037239075), ('guinea-pig', 0.44381460547447205), ('catfish', 0.4427811801433563), ('bushel', 0.4417347311973572), ('skinned', 0.4359341561794281), ('bitches', 0.43352121114730835), ('ma

In [31]:
limpia_lematiza("Hola Amigo mio")

'Hola Amigo mio'